In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class QuotesLoginSpider(scrapy.Spider):
    # Name of your spider
    name = "quotes"

    # Url to start your spider from
    start_urls = [
        'https://quotes.toscrape.com/login'
    ]

    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata = {'username':'john', 'password':'secret'},
            callback = self.after_login
        )

    def after_login(self, response):
        # retourne une liste de tous les éléments de classe "quote" 
        quotes = response.css('div.quote')
        for quote in quotes:
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('span small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }

        try:
            next_page = response.css('li.next a').attrib['href']
        except:
            logging.info("No next page")
        else:
            yield response.follow(next_page, callback=self.after_login)


filename = 'all_the_quotes_with_login.json'

if filename in os.listdir("src/"):
    os.remove('src/'+filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'FEEDS':{
        'src/' + filename : {"format": "json"},
        }

    }
)

#Lancer le crawler utilisant la spider
process.crawl(QuotesLoginSpider)
process.start()

2024-11-12 12:39:06 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-11-12 12:39:06 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.15 3 Sep 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-11-12 12:39:06 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-12 12:39:06 [py.warnings] WARNING: c:\Users\Admin\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu